In [1]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

2024-08-10 10:21:44.603975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 10:21:44.751550: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-10 10:21:44.788854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-10 10:21:44.788873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [42]:
import keras
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os
import warnings
import csv
from datetime import datetime 

pd.set_option('display.max_rows', 50)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
print("Version de TensorFlow :", tf.__version__)

Version de TensorFlow : 2.10.1


In [4]:
print("Version de Keras :", keras.__version__)

Version de Keras : 2.10.0


In [43]:
projectPath = "/home/carolus/Documents/school/green_ia/"
fileNbr = '174' # numéro d'identification des csv à charger et générer
csvPath = projectPath + "data/" + fileNbr + "_openfoodfacts_00/" 

# récupérer la date du jour 
currentDateTime = datetime.now()
formattedDate = currentDateTime.strftime("%d/%m/%Y %H:%M:%S.%f")[:-3]
dateCode = currentDateTime.strftime('%d%m%Y%H%M%S') + f"{currentDateTime.microsecond // 1000:03d}"

In [44]:
def addLogs(logData):
    print(logData)
    with open(f"{projectPath}logs/{dateCode}_logs.txt", "a") as logFile:
        logFile.write(f'{logData}\n')

In [45]:
addLogs(f"start date: {formattedDate}")
addLogs(f"fileNbr: {fileNbr} \nprojectPath: {projectPath} \ncsvPath: {csvPath}")

start date: 10/08/2024 15:32:19.778
fileNbr: 174 
projectPath: /home/carolus/Documents/school/green_ia/ 
csvPath: /home/carolus/Documents/school/green_ia/data/174_openfoodfacts_00/


In [13]:
# compter les fichiers csv dans le dossier traité 
def countCsv(directory):
    addLogs("counting small csv file in specific folder")
    csvNbr = 0
    for csvFile in os.listdir(directory):
        if csvFile.endswith('.csv'):
            csvNbr += 1
    return csvNbr

In [14]:
def deleteFile(filePath):
    if os.path.exists(filePath):
        os.remove(filePath)
        print(f"fichier {filePath} supprimé")
    else:
        print(f"erreur, fichier {filePath} n'existe pas")

In [59]:
# retourne le % de NaN par colonne
def countNaN(df, currentCsv):
    with open(currentCsv, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        line_count = sum(1 for row in reader) - 1

    nanCount = df.isna().sum()

    for column in nanCount.index:
        print(f"{column}: {nanCount[column]} NaN sur {line_count} lignes \n")

In [70]:
# compte le nombre de csv dans dossier 00
csvNbr = countCsv(csvPath)
addLogs(f'count small csv step 00: {csvNbr}')

csvIterator = 1
while csvIterator <= csvNbr:
    currentCsv = f"{csvPath}{csvIterator}_openfoodfacts_01.csv"
    print(f"current csv: {currentCsv}")

    # contenu du csv dans un df 
    df = pd.read_csv(currentCsv)
    df = df.sample(frac=1).reset_index(drop=True)

    # supprime les col trop vides 
    colToDrop = ['countries', 'additives_old_n', 'ecoscore_extended_data',
                  'allergens_tags', 'brand_owner', 'nova_group', 
                  'categories_old', 'categories_tags', 'brand_owner_imported',
                  'ingredients_hierarchy', 'food_groups_tags', 
                  'nutriscore_tags', 'nutriscore_2021_tags']
    df = df.drop(columns=colToDrop)

    # affiche le nombre de NaN par colonne
    nanCounts = countNaN(df, currentCsv)
    print(nanCounts)

    # faire incrémenter la nomenclature du fichier une fois preprocesse 

    csvIterator+=1    


count small csv step 00: 1
current csv: /home/carolus/Documents/school/green_ia/data/174_openfoodfacts_00/1_openfoodfacts_01.csv
allergens_from_ingredients: 225 NaN sur 10000 lignes 

nutriscore_tags: 0 NaN sur 10000 lignes 

labels_old: 0 NaN sur 10000 lignes 

pnns_groups_1: 1 NaN sur 10000 lignes 

ecoscore_data: 9685 NaN sur 10000 lignes 

ingredients_tags: 8437 NaN sur 10000 lignes 

packaging: 9632 NaN sur 10000 lignes 

product_name: 0 NaN sur 10000 lignes 

ecoscore_tags: 4808 NaN sur 10000 lignes 

ingredients_from_or_that_may_be_from_palm_oil_n: 5 NaN sur 10000 lignes 

nutrient_levels_tags: 4 NaN sur 10000 lignes 

categories: 8437 NaN sur 10000 lignes 

nutriments: 1 NaN sur 10000 lignes 

nutriscore_2021_tags: 1 NaN sur 10000 lignes 

ecoscore_score: 9743 NaN sur 10000 lignes 

labels_tags: 7 NaN sur 10000 lignes 

None


In [71]:
df.head(20)

,allergens_from_ingredients,nutriscore_tags,labels_old,pnns_groups_1,ecoscore_data,ingredients_tags,packaging,product_name,ecoscore_tags,ingredients_from_or_that_may_be_from_palm_oil_n,nutrient_levels_tags,categories,nutriments,nutriscore_2021_tags,ecoscore_score,labels_tags
0,roboto-app,100g,"['unknown', 'missing-category']",1707632985.0,NaN,NaN,NaN,3.327273e+12,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,4,"{'2021': {'nutriscore_computed': 0, 'nutrients...",NaN,[]
1,roboto-app,100g,"['unknown', 'missing-category']",1707633153.0,NaN,NaN,NaN,6.620002e+10,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,6,"{'2021': {'category_available': 0, 'data': {'s...",NaN,[]
2,kiliweb,100g,"['unknown', 'missing-category']",1707633103.0,NaN,NaN,NaN,8.887290e+12,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,5,"{'2021': {'grade': 'unknown', 'category_availa...",NaN,[]
3,barijaona,100g,"['unknown', 'missing-association']",1707633151.0,NaN,NaN,NaN,9.501100e+12,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,6,"{'2021': {'nutriscore_applicable': 1, 'nutrien...",NaN,[]
4,org-monoprix,100g,"['unknown', 'missing-category']",1707632786.0,NaN,4.0,NaN,3.350030e+12,on,"['en:no-packaging-data', 'en:ingredients-perce...",['unknown'],7.0,5,"{'2023': {'category_available': 0, 'data': {'e...",100g,[]
5,teolemon,100g,"['unknown', 'missing-association']",1707632952.0,NaN,NaN,NaN,8.412901e+12,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['b'],NaN,7,"{'2021': {'nutriscore_computed': 1, 'nutriscor...",NaN,[]
6,naruyoko,100g,"['unknown', 'missing-category']",1707633100.0,NaN,NaN,NaN,2.631733e+12,NaN,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,5,"{'2021': {'grade': 'unknown', 'data': {'fiber'...",NaN,[]
7,musarana,100g,"['unknown', 'missing-category']",1707632956.0,NaN,NaN,NaN,2.325616e+12,on,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,7,"{'2021': {'grade': 'unknown', 'category_availa...",NaN,[]
8,thaialagata,100g,"['unknown', 'missing-category']",1707632809.0,NaN,NaN,NaN,2.631803e+12,on,"['en:no-packaging-data', 'en:ecoscore-extended...",['unknown'],NaN,6,"{'2021': {'category_available': 0, 'data': {'e...",NaN,[]
9,danizavtz,100g,"['unknown', 'missing-category']",1707633107.0,NaN,10.0,NaN,3.350033e+12,on,"['en:no-packaging-data', 'en:ingredients-perce...",['unknown'],29.0,13,"{'2021': {'data': {'proteins': 8.3, 'energy': ...",100g,[]


In [63]:
print(df)

                             allergens_from_ingredients nutriscore_tags  \
0     ['en:to-be-completed', 'en:nutrition-facts-com...               0   
1     ['en:to-be-completed', 'en:nutrition-facts-com...               7   
2     ['en:to-be-completed', 'en:nutrition-facts-com...               0   
3     ['en:to-be-completed', 'en:nutrition-facts-to-...               0   
4     ['en:to-be-completed', 'en:nutrition-facts-com...               0   
...                                                 ...             ...   
9995                                        itsjustruby            100g   
9996                                           sebleouf            100g   
9997                                             tracky            100g   
9998                                         laeticoeur            100g   
9999                                mylabelcontributors            100g   

                           labels_old           pnns_groups_1  \
0                          1570305

In [76]:
toto = pd.read_csv('/home/carolus/Documents/school/green_ia/data/174_openfoodfacts_00/1_openfoodfacts_01.csv')
toto.tail(50)
# afficher contenu d'une colonne
#print(df['countries'].sample(n=18, random_state=50))

,allergens_from_ingredients,nutriscore_tags,labels_old,categories_old,pnns_groups_1,ecoscore_data,brand_owner_imported,ingredients_tags,packaging,ingredients_hierarchy,...,nutrient_levels_tags,allergens_tags,ecoscore_extended_data,categories,nutriments,nutriscore_2021_tags,additives_old_n,ecoscore_score,labels_tags,countries
9950,teolemon,100g,"['meat', 'known']",NaN,1707633307.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],en:poultry,NaN,NaN,5,"{'2021': {'nutriscore_computed': 0, 'nutrients...",NaN,NaN,[],NaN
9951,thaialagata,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,6,"{'2021': {'category_available': 0, 'data': {'e...",NaN,NaN,[],NaN
9952,thaialagata,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,7,"{'2023': {'category_available': 0, 'data': {'i...",NaN,NaN,[],NaN
9953,kiliweb,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,5,"{'2023': {'nutriscore_computed': 0, 'nutriscor...",NaN,NaN,[],NaN
9954,thaialagata,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,6,"{'2023': {'nutriscore_applicable': 0, 'nutrien...",NaN,NaN,[],NaN
9955,thaialagata,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,8,"{'2023': {'grade': 'unknown', 'data': {'is_bev...",NaN,NaN,[],NaN
9956,thaialagata,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,6,"{'2023': {'nutrients_available': 1, 'nutriscor...",NaN,NaN,[],NaN
9957,teolemon,100g,"['meat', 'known']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],en:poultry,NaN,NaN,5,"{'2023': {'nutriscore_computed': 0, 'nutrients...",NaN,NaN,[],NaN
9958,roboto-app,100g,"['unknown', 'missing-category']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['unknown'],NaN,NaN,NaN,4,"{'2021': {'grade': 'unknown', 'data': {'is_bev...",NaN,NaN,[],NaN
9959,thaialagata,100g,"['meat', 'known']",NaN,1707633308.0,NaN,NaN,NaN,NaN,NaN,...,['d'],en:meat-other-than-poultry,NaN,NaN,10,"{'2021': {'category_available': 1, 'data': {'n...",NaN,NaN,[],NaN
